## SPAM MESSAGE

In [7]:
def evaluate_email_with_human_check(state: EmailState) -> EmailState:
    """Evaluate email classification with a human confirmation step for spam."""
    # Check if the email meets spam criteria
    spam_detected = state["is_blacklisted"] or state["spam_score"] > 0.8 or state["is_malicious_attachment"]
    if spam_detected:
        print("\n⚠️ Potential Spam Detected ⚠️")
        print(f"Subject: {state['subject']}")
        print(f"Body: {state['body']}")
        print(f"Spam Score: {state['spam_score']:.2f}")

        # Ask user for confirmation
        human_input = input("Do you want to classify this email as spam? (yes/no): ").strip().lower()
        if human_input == "yes":
            state["classification"] = "spam"
            state["human_decision"] = f"Classified as spam by human (Score: {state['spam_score']:.2f})."
        else:
            state["classification"] = "not_spam"
            state["human_decision"] = (
                f"Originally detected as spam (Score: {state['spam_score']:.2f}), "
                "but user declared it as not spam."
            )
    elif state["is_whitelisted"]:
        state["classification"] = "not_spam"
        state["human_decision"] = "Automatically classified as not spam (Whitelisted sender)."
    else:
        state["classification"] = "quarantine"
        state["human_decision"] = "Automatically quarantined for further review."
    return state

def display_final_state(state: EmailState):
    """Display the final processed state in a clear format."""
    print("\n✅ Final Processed State:")
    print(f"  Sender: {state['sender']}")
    print(f"  Subject: {state['subject']}")
    print(f"  Body:\n    {state['body']}")
    print(f"  Attachments: {', '.join(state['attachments'])}")
    print(f"  Whitelisted: {state['is_whitelisted']}")
    print(f"  Blacklisted: {state['is_blacklisted']}")
    print(f"  Spam Score: {state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {state['is_malicious_attachment']}")
    print(f"  Classification: {state['classification']}")
    print(f"  Human Decision: {state['human_decision']}")

    # Additional spam detection details if relevant
    if state["classification"] == "spam":
        print(f"\n🔍 Spam Detection Details:\n  - Spam Score: {state['spam_score']:.2f}")
    elif state["classification"] == "not_spam" and "detected as spam" in state["human_decision"]:
        print(f"\n🔍 User Decision Details:\n  - Original Spam Score: {state['spam_score']:.2f}")
        print(f"  - User declared this email as not spam.")

# ============================================================
# MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    sender = "unknown@spamsite.com"
    subject = "Congratulations! You've won a free iPhone!"
    body = (
        "Dear user,\n\n"
        "You have been selected as a winner in our exclusive giveaway! "
        "Claim your free iPhone now by clicking the link below.\n\n"
        "Hurry, this offer expires soon!\n\n"
        "Best regards,\n"
        "Spammy Promotions Team"
    )
    attachments = ["offer.zip"]

    # Initial state
    initial_state = EmailState(
        messages=[],
        sender=sender,
        subject=subject,
        body=body,
        attachments=attachments,
        is_whitelisted=False,
        is_blacklisted=False,
        spam_score=0.0,
        content_category="",
        is_malicious_attachment=False,
        is_quarantined=False,
        human_decision="",
        classification=""
    )

    print("\n📧 Initial Email State:")
    print(f"  Sender: {initial_state['sender']}")
    print(f"  Subject: {initial_state['subject']}")
    print(f"  Body:\n    {initial_state['body']}")
    print(f"  Attachments: {', '.join(initial_state['attachments'])}")
    print(f"  Whitelisted: {initial_state['is_whitelisted']}")
    print(f"  Blacklisted: {initial_state['is_blacklisted']}")
    print(f"  Spam Score: {initial_state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {initial_state['is_malicious_attachment']}")
    print(f"  Classification: {initial_state['classification']}")

    print("\n🚀 Processing Email...\n")

    # Process the email
    state = process_email(initial_state)
    state = evaluate_email_with_human_check(state)

    # Display the final state
    display_final_state(state)


INFO:root:Tool started with input: {'subject': "Congratulations! You've won a free iPhone!", 'body': 'Dear user,\n\nYou have been selected as a winner in our exclusive giveaway! Claim your free iPhone now by clicking the link below.\n\nHurry, this offer expires soon!\n\nBest regards,\nSpammy Promotions Team'}
INFO:root:Tool ended with output: 0.9
INFO:root:Tool started with input: {'files': ['offer.zip']}
INFO:root:Malicious file detected: ['offer.zip']
INFO:root:Tool ended with output: True



📧 Initial Email State:
  Sender: unknown@spamsite.com
  Subject: Congratulations! You've won a free iPhone!
  Body:
    Dear user,

You have been selected as a winner in our exclusive giveaway! Claim your free iPhone now by clicking the link below.

Hurry, this offer expires soon!

Best regards,
Spammy Promotions Team
  Attachments: offer.zip
  Whitelisted: False
  Blacklisted: False
  Spam Score: 0.00
  Malicious Attachments: False
  Classification: 

🚀 Processing Email...


⚠️ Potential Spam Detected ⚠️
Subject: Congratulations! You've won a free iPhone!
Body: Dear user,

You have been selected as a winner in our exclusive giveaway! Claim your free iPhone now by clicking the link below.

Hurry, this offer expires soon!

Best regards,
Spammy Promotions Team
Spam Score: 0.90



✅ Final Processed State:
  Sender: unknown@spamsite.com
  Subject: Congratulations! You've won a free iPhone!
  Body:
    Dear user,

You have been selected as a winner in our exclusive giveaway! Claim your free iPhone now by clicking the link below.

Hurry, this offer expires soon!

Best regards,
Spammy Promotions Team
  Attachments: offer.zip
  Whitelisted: False
  Blacklisted: True
  Spam Score: 0.90
  Malicious Attachments: True
  Classification: not_spam
  Human Decision: Originally detected as spam (Score: 0.90), but user declared it as not spam.

🔍 User Decision Details:
  - Original Spam Score: 0.90
  - User declared this email as not spam.


## NON SPAM

In [ ]:
def evaluate_email_with_human_check(state: EmailState) -> EmailState:
    """Evaluate email classification with a human confirmation step for spam."""
    # Check if the email meets spam criteria
    spam_detected = state["is_blacklisted"] or state["spam_score"] > 0.8 or state["is_malicious_attachment"]
    if spam_detected:
        print("\n⚠️ Potential Spam Detected ⚠️")
        print(f"Subject: {state['subject']}")
        print(f"Body: {state['body']}")
        print(f"Spam Score: {state['spam_score']:.2f}")

        # Ask user for confirmation
        human_input = input("Do you want to classify this email as spam? (yes/no): ").strip().lower()
        if human_input == "yes":
            state["classification"] = "spam"
            state["human_decision"] = f"Classified as spam by human (Score: {state['spam_score']:.2f})."
        else:
            state["classification"] = "not_spam"
            state["human_decision"] = (
                f"Originally detected as spam (Score: {state['spam_score']:.2f}), "
                "but user declared it as not spam."
            )
    elif state["is_whitelisted"]:
        state["classification"] = "not_spam"
        state["human_decision"] = "Automatically classified as not spam (Whitelisted sender)."
    else:
        state["classification"] = "quarantine"
        state["human_decision"] = "Automatically quarantined for further review."
    return state

def display_final_state(state: EmailState):
    """Display the final processed state in a clear format."""
    print("\n✅ Final Processed State:")
    print(f"  Sender: {state['sender']}")
    print(f"  Subject: {state['subject']}")
    print(f"  Body:\n    {state['body']}")
    print(f"  Attachments: {', '.join(state['attachments'])}")
    print(f"  Whitelisted: {state['is_whitelisted']}")
    print(f"  Blacklisted: {state['is_blacklisted']}")
    print(f"  Spam Score: {state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {state['is_malicious_attachment']}")
    print(f"  Classification: {state['classification']}")
    print(f"  Human Decision: {state['human_decision']}")

    # Additional spam detection details if relevant
    if state["classification"] == "spam":
        print(f"\n🔍 Spam Detection Details:\n  - Spam Score: {state['spam_score']:.2f}")
    elif state["classification"] == "not_spam" and "detected as spam" in state["human_decision"]:
        print(f"\n🔍 User Decision Details:\n  - Original Spam Score: {state['spam_score']:.2f}")
        print(f"  - User declared this email as not spam.")

# ============================================================
# MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    sender = "colleague@company.com"
    subject = "Meeting Agenda for Tomorrow"
    body = (
        "Dear Team,\n\n"
        "I hope this email finds you well.\n\n"
        "Please find attached the agenda for tomorrow’s meeting scheduled at 10:00 AM in the conference room. "
        "Let me know if you have any additional topics to include.\n\n"
        "Looking forward to your input.\n\n"
        "Best regards,\n"
        "John Smith\n"
        "Manager, Operations"
    )
    attachments = ["agenda.pdf"]

    # Initial state
    initial_state = EmailState(
        messages=[],
        sender=sender,
        subject=subject,
        body=body,
        attachments=attachments,
        is_whitelisted=True,
        is_blacklisted=False,
        spam_score=0.0,
        content_category="",
        is_malicious_attachment=False,
        is_quarantined=False,
        human_decision="",
        classification=""
    )

    print("\n📧 Initial Email State:")
    print(f"  Sender: {initial_state['sender']}")
    print(f"  Subject: {initial_state['subject']}")
    print(f"  Body:\n    {initial_state['body']}")
    print(f"  Attachments: {', '.join(initial_state['attachments'])}")
    print(f"  Whitelisted: {initial_state['is_whitelisted']}")
    print(f"  Blacklisted: {initial_state['is_blacklisted']}")
    print(f"  Spam Score: {initial_state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {initial_state['is_malicious_attachment']}")
    print(f"  Classification: {initial_state['classification']}")

    print("\n🚀 Processing Email...\n")

    # Process the email
    state = process_email(initial_state)
    state = evaluate_email_with_human_check(state)

    # Display the final state
    display_final_state(state)


INFO:root:Tool started with input: {'subject': 'Meeting Agenda for Tomorrow', 'body': 'Dear Team,\n\nI hope this email finds you well.\n\nPlease find attached the agenda for tomorrow’s meeting scheduled at 10:00 AM in the conference room. Let me know if you have any additional topics to include.\n\nLooking forward to your input.\n\nBest regards,\nJohn Smith\nManager, Operations'}



📧 Initial Email State:
  Sender: colleague@company.com
  Subject: Meeting Agenda for Tomorrow
  Body:
    Dear Team,

I hope this email finds you well.

Please find attached the agenda for tomorrow’s meeting scheduled at 10:00 AM in the conference room. Let me know if you have any additional topics to include.

Looking forward to your input.

Best regards,
John Smith
Manager, Operations
  Attachments: agenda.pdf
  Whitelisted: True
  Blacklisted: False
  Spam Score: 0.00
  Malicious Attachments: False
  Classification: 

🚀 Processing Email...



INFO:root:Tool ended with output: 0.0
INFO:root:Tool started with input: {'files': ['agenda.pdf']}
INFO:root:Tool ended with output: False



✅ Final Processed State:
  Sender: colleague@company.com
  Subject: Meeting Agenda for Tomorrow
  Body:
    Dear Team,

I hope this email finds you well.

Please find attached the agenda for tomorrow’s meeting scheduled at 10:00 AM in the conference room. Let me know if you have any additional topics to include.

Looking forward to your input.

Best regards,
John Smith
Manager, Operations
  Attachments: agenda.pdf
  Whitelisted: True
  Blacklisted: False
  Spam Score: 0.00
  Malicious Attachments: False
  Classification: not_spam
  Human Decision: Automatically classified as not spam (Whitelisted sender).


In [ ]:
def evaluate_email_with_human_check(state: EmailState) -> EmailState:
    """Evaluate email classification with a human confirmation step for spam."""
    # Check if the email meets spam criteria
    spam_detected = state["is_blacklisted"] or state["spam_score"] > 0.8 or state["is_malicious_attachment"]
    if spam_detected:
        print("\n⚠️ Potential Spam Detected ⚠️")
        print(f"Subject: {state['subject']}")
        print(f"Body: {state['body']}")
        print(f"Spam Score: {state['spam_score']:.2f}")

        # Ask user for confirmation
        human_input = input("Do you want to classify this email as spam? (yes/no): ").strip().lower()
        if human_input == "yes":
            state["classification"] = "spam"
            state["human_decision"] = f"Classified as spam by human (Score: {state['spam_score']:.2f})."
        else:
            state["classification"] = "not_spam"
            state["human_decision"] = (
                f"Originally detected as spam (Score: {state['spam_score']:.2f}), "
                "but user declared it as not spam."
            )
    elif state["is_whitelisted"]:
        state["classification"] = "not_spam"
        state["human_decision"] = "Automatically classified as not spam (Whitelisted sender)."
    else:
        state["classification"] = "quarantine"
        state["human_decision"] = "Automatically quarantined for further review."
    return state

def display_final_state(state: EmailState):
    """Display the final processed state in a clear format."""
    print("\n✅ Final Processed State:")
    print(f"  Sender: {state['sender']}")
    print(f"  Subject: {state['subject']}")
    print(f"  Body:\n    {state['body']}")
    print(f"  Attachments: {', '.join(state['attachments'])}")
    print(f"  Whitelisted: {state['is_whitelisted']}")
    print(f"  Blacklisted: {state['is_blacklisted']}")
    print(f"  Spam Score: {state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {state['is_malicious_attachment']}")
    print(f"  Classification: {state['classification']}")
    print(f"  Human Decision: {state['human_decision']}")

    # Additional spam detection details if relevant
    if state["classification"] == "spam":
        print(f"\n🔍 Spam Detection Details:\n  - Spam Score: {state['spam_score']:.2f}")
    elif state["classification"] == "not_spam" and "detected as spam" in state["human_decision"]:
        print(f"\n🔍 User Decision Details:\n  - Original Spam Score: {state['spam_score']:.2f}")
        print(f"  - User declared this email as not spam.")

# ============================================================
# MAIN EXECUTION
# ============================================================

if __name__ == "__main__":
    sender = "promotions@shoppingdeals.com"
    subject = "Limited Time Offer on Electronics"
    body = (
        "Dear Valued Customer,\n\n"
        "We’re excited to bring you an exclusive deal on the latest electronics! "
        "Enjoy up to 50% off on top brands for a limited time only.\n\n"
        "Visit our website now and take advantage of these amazing discounts before they’re gone!\n\n"
        "Best regards,\n"
        "The Shopping Deals Team"
    )
    attachments = []

    # Initial state
    initial_state = EmailState(
        messages=[],
        sender=sender,
        subject=subject,
        body=body,
        attachments=attachments,
        is_whitelisted=False,
        is_blacklisted=False,
        spam_score=0.6,  # Moderate spam score
        content_category="Promotions",
        is_malicious_attachment=False,
        is_quarantined=False,
        human_decision="",
        classification=""
    )

    print("\n📧 Initial Email State:")
    print(f"  Sender: {initial_state['sender']}")
    print(f"  Subject: {initial_state['subject']}")
    print(f"  Body:\n    {initial_state['body']}")
    print(f"  Attachments: {', '.join(initial_state['attachments'])}")
    print(f"  Whitelisted: {initial_state['is_whitelisted']}")
    print(f"  Blacklisted: {initial_state['is_blacklisted']}")
    print(f"  Spam Score: {initial_state['spam_score']:.2f}")
    print(f"  Malicious Attachments: {initial_state['is_malicious_attachment']}")
    print(f"  Classification: {initial_state['classification']}")

    print("\n🚀 Processing Email...\n")

    # Process the email
    state = process_email(initial_state)
    state = evaluate_email_with_human_check(state)

    # Display the final state
    display_final_state(state)



INFO:root:Tool started with input: {'subject': 'Limited Time Offer on Electronics', 'body': 'Dear Valued Customer,\n\nWe’re excited to bring you an exclusive deal on the latest electronics! Enjoy up to 50% off on top brands for a limited time only.\n\nVisit our website now and take advantage of these amazing discounts before they’re gone!\n\nBest regards,\nThe Shopping Deals Team'}



📧 Initial Email State:
  Sender: promotions@shoppingdeals.com
  Subject: Limited Time Offer on Electronics
  Body:
    Dear Valued Customer,

We’re excited to bring you an exclusive deal on the latest electronics! Enjoy up to 50% off on top brands for a limited time only.

Visit our website now and take advantage of these amazing discounts before they’re gone!

Best regards,
The Shopping Deals Team
  Attachments: 
  Whitelisted: False
  Blacklisted: False
  Spam Score: 0.60
  Malicious Attachments: False
  Classification: 

🚀 Processing Email...

